In [38]:
#hide
#import the necessary packages
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot as ply
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import altair as alt
import numpy as np
from matplotlib.ticker import FormatStrFormatter
#from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as wid
from IPython.core.display import display,HTML
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')

In [39]:
#hide_input
# Covid 19 Global Dashboard title formatting with markdown
heading= '''
    <div>

    <link rel="stylesheet"
    href="https://fonts.googleapis.com/css2?family=Suez+One">
    <div style="font-family: 'Suez One';font-size:70px; background-color:white; color:black"><center>Covid-19 Global Dashboard</center></div>

    </div>
    '''
html = HTML(heading.format())
display(html)
# Reading the world data from covid.ourworldindata.org
# The data is cumulative, so the total_cases for a day sums up the cases per day 
df1= pd.read_csv("https://covid.ourworldindata.org/data/ecdc/full_data.csv")

#Repeating the same for India 
#Reading the data from covid19india.org
data=pd.read_csv("https://api.covid19india.org/csv/latest/states.csv")
#The data contains an unassigned state which is removed
df8 = data[data['State'] != 'State Unassigned']
#Removing unnecessary columns
df8=df8[['Date','State','Confirmed','Recovered','Deceased']]
#Renaming the columns since the hover data is based on the column names
df8.columns=['Date','State','Confirmed Cases','Recovered','Deceased']
#Getting a list of all dates
dates=list(set(df8.Date))
dates.sort()
#Getting a list of all States
states=list(set(df8.State))
#Findingtodays date
dt_tday=dates[-1]
#Finding yesterdays date
dt_yday=dates[-2]
#Getting todays data for all states available
dfc=df8[df8['Date']==dt_tday]
#This is done for compatibility of state names with the geojson 
dfc=dfc.replace("Andaman and Nicobar Islands",'Andaman & Nicobar')
#Saving the data for India
India=dfc[dfc['State'] == 'India']
#Removing India's data from the dataset 
dfc = dfc[dfc['State'] != 'India']

In [42]:
#hide

def state(State):
            day_data={}
            temp=df8[df8['State']==State]
            day_data[f'{State} date']=temp['Date']
            day_data[f'{State} cases']=temp[['Confirmed Cases']].diff(axis=0).fillna(0).astype(int)
            day_data[f'{State} recovered']=temp[['Recovered']].diff(axis=0).fillna(0).astype(int)
            day_data[f'{State} deaths']=temp[['Deceased']].diff(axis=0).fillna(0).astype(int)


            fig = plt.figure(figsize = (8,9))
            ax = fig.add_subplot(311)
            ef=pd.DataFrame()
            ef['date']=day_data[f'{State} date'].astype('datetime64[ns]')
            ef['cases']=day_data[f'{State} cases']
            ax.bar(ef.date,ef.cases,color = '#007acc',alpha=0.3)
            ax.plot(ef.date,ef.cases,marker='o',color='#007acc')
            ax.set_title(f'{State}\'s daily case count', fontsize = 23);
            ax.xaxis.set_major_locator(mdates.WeekdayLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
            ax.tick_params(rotation=60)
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax1 = fig.add_subplot(312)
            ef['deaths']=day_data[f'{State} deaths']
            ax1.bar(ef.date,ef.deaths,color = '#007acc',alpha=0.3)
            ax1.plot(ef.date,ef.deaths,marker='o',color='#007acc')
            ax1.set_title(f'{State}\'s daily death count',fontsize=23)
            ax1.xaxis.set_major_locator(mdates.WeekdayLocator())
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
            ax1.tick_params(rotation=60)
            ax1.spines['right'].set_visible(False)
            ax1.spines['top'].set_visible(False)
            ax2 = fig.add_subplot(313)
            ef['recovered']=day_data[f'{State} recovered']
            ax2.bar(ef.date,ef.recovered,color = '#007acc',alpha=0.3)
            ax2.plot(ef.date,ef.recovered,marker='o',color='#007acc')
            ax2.set_title(f'{State}\'s daily recovery count',fontsize=23)
            ax2.xaxis.set_major_locator(mdates.WeekdayLocator())
            ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
            ax2.tick_params(rotation=60)
            ax2.spines['right'].set_visible(False)
            ax2.spines['top'].set_visible(False)
            fig.tight_layout()

            ply_fig = tls.mpl_to_plotly(fig)
            ply(ply_fig);

            df9=dfc.replace("Andaman & Nicobar",'Andaman and Nicobar Islands')
            c=df9[dfc['State']==State]['Confirmed Cases'].iloc[0]
            d=df9[dfc['State']==State]['Deceased'].iloc[0]
            r=df9[dfc['State']==State]['Recovered'].iloc[0]

            heading7 = '''
            <!-- #######  HTML!! #########-->
            <h1 style="text-align: center; font-size:30px;">India's Statistics</h1>
            <p style="text-align: center; font-size:20px;"> Confirmed Cases: <span style="color: #ff0000; font-size:20px;">{c}</span> </p>
            <p style="text-align: center; font-size:20px;"> Confirmed Deaths: <span style="color: #ff0000; font-size:20px;">{d}</span> </p>
            <p style="text-align: center; font-size:20px;"> Recovered: <span style="color: #ff0000; font-size:20px;">{r}</span> </p>
            <p style="text-align: center; font-size:20px;"> Current Cases: <span style="color: #ff0000; font-size:20px;">{cc}</span> </p>
            <p style="text-align: center; font-size:20px;"> Fatality Rate: <span style="color: #ff0000; font-size:20px;">{f} %</span> </p>
            <p style="text-align: center; font-size:20px;"> Recovery Rate: <span style="color: #ff0000; font-size:20px;">{rr} %</span> </p>
            <p><br></p>
            <p><br></p>
            '''
            cc=int(c-d-r)
            f=round((d/c)*100,2)
            rr=round((r/c)*100,2)

            html = HTML(heading7.format(c=int(c),d=int(d),r=int(r),cc=int(c-d-r),f=round((d/c)*100,2),rr=round((r/c)*100,2)));
            display(html);


def dashboards(Dashboards='Global'):
    if Dashboards == "Global":      
            #Getting the unique dates present in the dataset
            dates=list(set(df1.date))
            #Sorting the dates to get the most recent date
            #If we use datetime to get the current day, at 12am there will be no data to show as the data for the day would not be updated yet
            dates.sort()
            dt_tday=dates[-1]
            #Getting the data for the most recent date 
            td = df1[df1['date'] ==  dt_tday]
            #Resetting the index
            td=td.reset_index(drop=True)
            #This the text used for the hover data, anything to be added to it should be done here
            #Add a '<br>' after each name and data to move to the next line
            txt= ' Country: ' + td['location'].astype(str) + '<br>' + ' Cases: ' + td['total_cases'].astype(str) + '<br>'+ ' Deaths: ' + td['total_deaths'].astype(str)  
            #The country names are converted to lowercase for compatibility with the inbuilt location names in graph_object plotting
            td['location']=td['location'].str.lower()
            #Saving the world data from the dataset 
            world= td[td['location']=='world']
            #Removing the world data from the dataset
            td=td[td['location']!='world']
            
            
            spacing = '''
            <!-- #######  HTML!! #########-->
            <p><br></p>
            <p><br></p>
            '''
            html = HTML(spacing.format())
            display(html);
            
            #This is to plot the global map
            fig1 = go.Figure(data=go.Choropleth(
                    locations = td['location'],
                    locationmode='country names',
                    z = td['total_cases'],   #Colour of the countries are based on this value
                    colorbar_title = "Total Cases",
                    text = txt, #Hoverdata 
                    colorbar = {'len':0.75,'lenmode':'fraction'},
                    marker_line_color='black',
                    marker_line_width=0.5))

            fig1.update_layout(
                width = 1100,
                height = 600,
                title={                           #This is to set the size and location of the title of the map 
                'text':'Global Covid Data',
                'y':1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font':{ 'color':'Black', 'size':30}},
                geo=dict(                         #Removing the frame borders and giving the projection type
                showframe=False,
                showcoastlines=True,
                ),
                margin= dict(
                    l= 0,
                    r= 0,
                    b= 0,
                    t= 0,
                    pad=0
                ))
            
            ply(fig1);
            
            c=int(world['total_cases'].iloc[0])
            d=int(world['total_deaths'].iloc[0])
            f=round((d/c)*100,2)
            
            #Printing out the statistics of the world for the most recent date
            heading3 = '''
            <!-- #######  HTML!! #########-->
            <h1 style="text-align: center; font-size:30px;">World Statistics</h1>
            <p style="text-align: center; font-size:20px;"> Confirmed Cases: <span style="color: #ff0000; font-size:20px;">{c}</span> </p>
            <p style="text-align: center; font-size:20px;"> Confirmed Deaths: <span style="color: #ff0000; font-size:20px;">{d}</span> </p>
            <p style="text-align: center; font-size:20px;"> Fatality Rate: <span style="color: #ff0000; font-size:20px;">{f} %</span> </p>
            <p><br></p>
            <p><br></p>
            <p><br></p>
            <h1 style="text-align: center; font-size:30px;">Country-wise statistics</h1>
            '''
            html = HTML(heading3.format(c=c,d=d,f=f))
            display(html);
            
            #Getting a list of all the unique contries present after removing 'World'
            countries=list(set(df1.location))
            countries.remove('World')
            countries.remove('International')
            countries.sort()
            countries.remove('India')
            countries.insert(0,'India');
            
            def country(Country='India'):
                                
                    spacing = '''
                    <!-- #######  HTML!! #########-->
                    <p><br></p>
                    '''
                    html = HTML(spacing.format())
                    display(html);
                    
                    #Checking if there is a country selected and if there is, give its information
                    #This is to pull out the day and total cases for the selected country 

                    day_data={}
                    temp=df1[df1['location']==Country]
                    day_data[f'{Country} date']=temp['date']
                    day_data[f'{Country} cases']=temp[['total_cases']].diff(axis=0).fillna(0).astype(int)
                    day_data[f'{Country} deaths']=temp[['total_deaths']].diff(axis=0).fillna(0).astype(int)
                    #Plot used for the Univ.ai Covid dashboard question
                    fig = plt.figure(figsize = (15,9))
                    ax = fig.add_subplot(211)
                    ef=pd.DataFrame()
                    ef['date']=day_data[f'{Country} date'].astype('datetime64[ns]')
                    ef['cases']=day_data[f'{Country} cases']
                    ax.bar(ef.date,ef.cases,color = '#007acc',alpha=0.3)
                    ax.plot(ef.date,ef.cases,marker='o',color='#007acc')
                    
                    ax.set_title(f'{Country}\'s daily case count',fontsize=23)
                    ax.xaxis.set_major_locator(mdates.WeekdayLocator())
                    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
                    ax.tick_params(rotation=60)
                    ax.spines['right'].set_visible(False)
                    ax.spines['top'].set_visible(False)
                    
                    ax1 = fig.add_subplot(212)
                    ef['deaths']=day_data[f'{Country} deaths']
                    ax1.bar(ef.date,ef.deaths,color = '#007acc',alpha=0.3)
                    ax1.plot(ef.date,ef.deaths,marker='o',color='#007acc')
                    ax1.set_title(f'{Country}\'s daily death count',fontsize=23)
                    ax1.xaxis.set_major_locator(mdates.WeekdayLocator())
                    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b%d'))
                    ax1.tick_params(rotation=60)
                    ax1.spines['right'].set_visible(False)
                    ax1.spines['top'].set_visible(False)
                    fig.tight_layout()
                    ply_fig = tls.mpl_to_plotly(fig)
                    ply(ply_fig);

                    # Printing out information for the country for the most recent date
                    c=int(td[td['location']==Country.lower()]['total_cases'].iloc[0])
                    d=int(td[td['location']==Country.lower()]['total_deaths'].iloc[0])
                    f=round((d/c)*100,2)

                    #Printing out the statistics of the world for the most recent date
                    heading4 = '''
                    <!-- #######  HTML!! #########-->
                    <h1 style="text-align: center; font-size:30px;">{Country}'s Statistics</h1>
                    <p style="text-align: center; font-size:20px;"> Confirmed Cases: <span style="color: #ff0000; font-size:20px;">{c}</span> </p>
                    <p style="text-align: center; font-size:20px;"> Confirmed Deaths: <span style="color: #ff0000; font-size:20px;">{d}</span> </p>
                    <p style="text-align: center; font-size:20px;"> Fatality Rate: <span style="color: #ff0000; font-size:20px;">{f} %</span> </p>
                    <p><br></p>
                    <p><br></p>
                    <h1 style="text-align: center;">Comparison of infection growth</h1>
                    '''
                    html = HTML(heading4.format(c=c,d=d,f=f,Country=Country));
                    display(html);

                    

            df = df1;
            df=df[(df.location!='World') & (df.location!='International')];
            df['date']=df['date'].astype('datetime64[ns]');

            def Compare(Countries='India'):
                subset_data=df[df['location'].isin(Countries)]

                total_cases_graph  =alt.Chart(subset_data).transform_filter(
                alt.datum.total_cases > 0  
                ).mark_line().encode(
                x=alt.X('date:T', title='Date',timeUnit='yearmonthdate', axis=alt.Axis(labelAngle=-45)),
                y=alt.Y('total_cases:Q',  title='Confirmed cases'),
                color='location',
                tooltip = ['location','total_cases'],
                ).properties(
                width=950,
                height=300
                ).configure_axis(
                labelFontSize=10,
                titleFontSize=15
                )
                display(total_cases_graph);  

            #The dropdown for selecting the country 
            w=wid.interact(country, Country=wid.Dropdown(options=countries,layout=wid.Layout(width='1000px',align='center')));
            display(w);

            temp=df.groupby('location').count().reset_index()['location'].tolist();
            wd=wid.interact(Compare, Countries=wid.SelectMultiple(options=temp,layout=wid.Layout(width='1000px',align='center'),value=['India']));
            display(wd);

    elif Dashboards == "India": 
        #Link to the geojson
        gj=r"https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
        fig2 = px.choropleth(
            dfc,
            geojson=gj,
            featureidkey='properties.ST_NM',
            locations='State',
            color='Confirmed Cases',
            color_continuous_scale="Blues",
            projection="mercator",
            hover_data=['State','Confirmed Cases','Deceased','Recovered']) #The data is pulled out from the dataframe dfc in this case
        fig2.update_geos(fitbounds="locations", visible=False)
        fig2.update_layout(
            autosize=False,
            width=1100,    #Here I am able to change the height and width of the graph unlike before
            height=700,
            title={
                'text': 'India',
                'y':1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',
            'font':{ 'color':'blue', 'size':30}}
        )

        ply(fig2)
        #Printing out India's stats 

        heading6 = '''
        <!-- #######  HTML!! #########-->
        <h1 style="text-align: center; font-size:30px;">India's Statistics</h1>
        <p style="text-align: center; font-size:20px;"> Confirmed Cases: <span style="color: #ff0000; font-size:20px;">{c}</span> </p>
        <p style="text-align: center; font-size:20px;"> Confirmed Deaths: <span style="color: #ff0000; font-size:20px;">{d}</span> </p>
        <p style="text-align: center; font-size:20px;"> Recovered: <span style="color: #ff0000; font-size:20px;">{r}</span> </p>
        <p style="text-align: center; font-size:20px;"> Current Cases: <span style="color: #ff0000; font-size:20px;">{cc}</span> </p>
        <p style="text-align: center; font-size:20px;"> Fatality Rate: <span style="color: #ff0000; font-size:20px;">{f} %</span> </p>
        <p style="text-align: center; font-size:20px;"> Recovery Rate: <span style="color: #ff0000; font-size:20px;">{rr} %</span> </p>
        <p><br></p>
        <p><br></p>
        '''
        c=India['Confirmed Cases'].iloc[0]
        d=India['Deceased'].iloc[0]
        r=India['Recovered'].iloc[0]
        cc=int(c-d-r)
        f=round((d/c)*100,2)
        rr=round((r/c)*100,2)

        html = HTML(heading6.format(c=int(c),d=int(d),r=int(r),cc=cc,f=f,rr=rr))
        display(html);

        #Removing India from the list of states
        states=list(dfc.sort_values(by=['Confirmed Cases','Deceased'], ascending=[False, False]).State)


        #Giving the information for each state similar to info for each country
        wa=wid.interact(state, State=wid.Dropdown(options=states,layout=wid.Layout(width='1000px',align='center')))
        display(wa);

            
def analysis(Analysis='Overview'):
    if Analysis == "Overview":
        heading2 = '''
        <!-- #######  HTML!! #########-->
        <link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Suez+One">
        <h1 style="font-family: 'Suez One';text-align: center;font-size:40px;">Dashboards</h1>
        '''
        html = HTML(heading2.format());
        display(html);
        w=wid.interact(dashboards, Dashboards=wid.Dropdown(options=["Global","India"],layout=wid.Layout(width='1000px',align='center')));
        display(w);
    
        


In [43]:
#hide_input
# Covid 19 Global Dashboard title formatting with markdown
heading1 = '''
<!-- #######  HTML!! #########-->
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Suez+One">
<h1 style="font-family:'Suez One';text-align: center; font-size:40px;">Analysis</h1>
'''
html = HTML(heading1.format())
display(html);
w=wid.interact(analysis, Analysis=wid.Dropdown(options=["Overview","Fatalities","Trend"],layout=wid.Layout(width='1000px',justify='center')));
display(w);

interactive(children=(Dropdown(description='Analysis', layout=Layout(width='1000px'), options=('Overview', 'Fa…

<function __main__.analysis>